In [1]:
import pandas as pd

df = pd.read_parquet('/drive2/tuandung/DevEval/Generations/codellama13b/LFCLF_2_embedding_dev_eval_codellama_CodeLlama-13b-Instruct-hf_40_full_clean_code_label.parquet')

In [7]:
df[df['label'] == 0]

,task_id,completion_id,num_tokens,generation,first_token_embedding,last_token_embedding,first_token_code_embedding,last_token_code_embedding,has_error,first_code_index,last_code_index,first_index,last_index,generated_ids,extracted_code,cleaned_code,label
0,boto.s3.bucket.Bucket.new_key,boto.s3.bucket.Bucket.new_key_0,357,"```Python\n def new_key(self, key_name=Non...","[-0.30078125, -0.302734375, 0.42578125, 1.0546...","[-0.042724609375, 0.69921875, 0.87890625, -0.5...","[0.443359375, -0.59375, 0.359375, 0.328125, -0...","[-0.0081787109375, 0.421875, 0.080078125, 0.53...",False,5,137,0,355,"[29871, 7521, 11980, 13, 1678, 822, 716, 29918...","def new_key(self, key_name=None):\n """"""...",Explanation:\n\nThe `new_key` function creates...,0
1,boto.s3.bucket.Bucket.new_key,boto.s3.bucket.Bucket.new_key_1,358,"Based on the provided context, here is the co...","[-0.30078125, -0.302734375, 0.42578125, 1.0546...","[-1.6484375, 0.65234375, 0.9296875, -1.8515625...","[0.8046875, -0.0302734375, 0.65234375, 0.58984...","[0.5859375, 0.25390625, 0.11865234375, 0.31835...",False,25,184,0,356,"[29871, 16564, 373, 278, 4944, 3030, 29892, 12...","def new_key(self, key_name=None):\n """"""\n ...",if key_name is None:\n # If no key ...,0
2,boto.s3.bucket.Bucket.new_key,boto.s3.bucket.Bucket.new_key_2,163,"```\ndef new_key(self, key_name=None):\n ""...","[-0.30078125, -0.302734375, 0.42578125, 1.0546...","[-0.302734375, -0.2373046875, 1.0625, -1.50781...","[0.7265625, -0.1162109375, 0.482421875, 0.6445...","[0.306640625, -0.29296875, 0.0152587890625, 0....",False,3,108,0,161,"[29871, 7521, 13, 1753, 716, 29918, 1989, 2989...","def new_key(self, key_name=None):\n """"""\n ...",The completed code defines the `new_key` metho...,0
3,boto.s3.bucket.Bucket.new_key,boto.s3.bucket.Bucket.new_key_3,162,"```\n def new_key(self, key_name=None):\n ...","[-0.30078125, -0.302734375, 0.42578125, 1.0546...","[-0.0419921875, 0.08251953125, -0.11279296875,...","[0.5078125, -0.5546875, 0.38671875, 0.30859375...","[0.3359375, -0.029052734375, -0.337890625, 0.5...",False,4,115,0,160,"[29871, 7521, 13, 1678, 822, 716, 29918, 1989,...","def new_key(self, key_name=None):\n """"""...",This function creates a new instance of the ke...,0
5,boto.s3.bucket.Bucket.new_key,boto.s3.bucket.Bucket.new_key_5,270,"Based on the context and the provided code, h...","[-0.30078125, -0.302734375, 0.42578125, 1.0546...","[0.059326171875, -0.2236328125, 0.11328125, -0...","[0.5546875, -0.51953125, 0.49609375, 0.1933593...","[0.5234375, 0.57421875, 0.26953125, 0.02258300...",False,30,159,0,268,"[29871, 16564, 373, 278, 3030, 322, 278, 4944,...","def new_key(self, key_name=None):\n """"""...",if key_name is None:\n rais...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17325,pythonforandroid.prerequisites.AutomakePrerequ...,pythonforandroid.prerequisites.AutomakePrerequ...,400,```Python\ndef darwin_installer():\n info(...,"[-0.52734375, -0.333984375, 0.44140625, 0.6640...","[-1.3359375, 0.396484375, -0.1708984375, 0.679...","[-0.171875, -0.1953125, 0.2099609375, 0.75, -0...","[0.2314453125, -0.283203125, 0.051025390625, 0...",False,4,397,0,398,"[29871, 7521, 11980, 13, 1753, 5424, 5080, 299...",def darwin_installer():\n info('Looking for...,info('Looking for a JDK 11 or higher insta...,0
17326,pythonforandroid.prerequisites.AutomakePrerequ...,pythonforandroid.prerequisites.AutomakePrerequ...,400,Here is the completed code for the `darwin_in...,"[-0.52734375, -0.333984375, 0.44140625, 0.6640...","[-0.142578125, 0.2236328125, 0.283203125, -0.6...","[-0.1337890625, -0.1220703125, 0.296875, 0.562...","[-0.0303955078125, 0.06591796875, -0.171875, -...",False,21,397,0,398,"[29871, 2266, 338, 278, 8676, 775, 363, 278, 4...",def darwin_installer(self):\n # Check if JD...,jdk_path = self._darwin_get_libexec_jdk_pa...,0
17327,pythonforandroid.prerequisites.AutomakePrerequ...,pythonforandroid.prerequisites.AutomakePrerequ...,400,```python\nimport os\nimport platform\nimport.

In [1]:
import pandas as pd

df = pd.read_parquet('/drive2/tuandung/DevEval/Generations/codellama13b/LFCLF_2_embedding_dev_eval_codellama_CodeLlama-13b-Instruct-hf_40_full.parquet')

In [3]:
df.iloc[0]['extracted_code']

"def spatial_match(password, _graphs=GRAPHS, _ranked_dictionaries=RANKED_DICTIONARIES):\n    matches = []\n    for graph_name, graph in _graphs.items():\n        for token in re.findall(graph['regex'], password):\n            for variation in variations(token):\n                rank_of_variation = rank_of(variation, graph_name)\n                if rank_of_variation > 0:\n                    matches.append({\n                        'pattern': 'spatial',\n                        'token': token,\n                        'graph': graph_name,\n                        'turns': graph['turns'],\n                        'shifted_count': graph['shifted_count'],\n                        'found_rank': rank_of_variation,\n                    })\n\n    # apply l33t match on the remaining tokens\n    for match in l33t_match(password, ranked_dictionaries=_ranked_dictionaries):\n        if match not in matches:\n            matches.append(match)\n\n    return sorted(matches, key=lambda x: (x['i'], x['

In [3]:
df['cleaned_code']

0            text = text.split()\n    ngrams = []\n    ...
1            text = text.split()\n    ngrams = []\n    ...
2            text = text.split(' ')\n    ngrams = set()...
3            text = text.split(' ')\n    ngrams = []\n ...
4            text = text.split()\n    ngrams = []\n    ...
                               ...                        
17325            other = copy.copy(self)\n        for k...
17326            other = copy.copy(self)\n        for k...
17327            other = copy.copy(self)\n        for k...
17328            other = copy.copy(self)\n        for k...
17329            other = copy.copy(self)\n        for k...
Name: cleaned_code, Length: 17330, dtype: object

In [5]:
df[df['label'] == 1]

,task_id,completion_id,num_tokens,generation,first_token_embedding,last_token_embedding,first_token_code_embedding,last_token_code_embedding,has_error,extracted_code,cleaned_code,label
10,boto.sdb.db.sequence.fib,boto.sdb.db.sequence.fib_0,59,"```Python\ndef fib(cv=1, lv=0):\n """"""The fi...","[-0.12158203125, 0.4765625, -0.1728515625, -0....","[-0.0810546875, 0.451171875, -0.015625, -0.034...","[-0.01123046875, 0.1240234375, -0.138671875, -...","[0.3515625, -0.0302734375, -0.01708984375, 0.1...",False,"def fib(cv=1, lv=0):\n """"""The fibonacci seq...",if cv is None:\n return 0\n retu...,1
11,boto.sdb.db.sequence.fib,boto.sdb.db.sequence.fib_1,59,"```Python\ndef fib(cv=1, lv=0):\n """"""The fi...","[-0.12158203125, 0.4765625, -0.1728515625, -0....","[-0.0810546875, 0.451171875, -0.015625, -0.034...","[-0.01123046875, 0.1240234375, -0.138671875, -...","[0.3515625, -0.0302734375, -0.01708984375, 0.1...",False,"def fib(cv=1, lv=0):\n """"""The fibonacci seq...",if cv is None:\n return 0\n retu...,1
12,boto.sdb.db.sequence.fib,boto.sdb.db.sequence.fib_2,59,"```Python\ndef fib(cv=1, lv=0):\n """"""The fi...","[-0.12158203125, 0.4765625, -0.1728515625, -0....","[-0.0810546875, 0.451171875, -0.015625, -0.034...","[-0.01123046875, 0.1240234375, -0.138671875, -...","[0.3515625, -0.0302734375, -0.01708984375, 0.1...",False,"def fib(cv=1, lv=0):\n """"""The fibonacci seq...",if cv is None:\n return 0\n retu...,1
13,boto.sdb.db.sequence.fib,boto.sdb.db.sequence.fib_3,59,"```Python\ndef fib(cv=1, lv=0):\n """"""The fi...","[-0.12158203125, 0.4765625, -0.1728515625, -0....","[-0.0810546875, 0.451171875, -0.015625, -0.034...","[-0.01123046875, 0.1240234375, -0.138671875, -...","[0.3515625, -0.0302734375, -0.01708984375, 0.1...",False,"def fib(cv=1, lv=0):\n """"""The fibonacci seq...",if cv is None:\n return 0\n retu...,1
14,boto.sdb.db.sequence.fib,boto.sdb.db.sequence.fib_4,158,The fibonacci sequence is a series of numbers ...,"[-0.0947265625, -0.083984375, -0.1572265625, 0...","[-0.052734375, 0.04736328125, 0.01214599609375...","[0.0032958984375, 0.1240234375, -0.1357421875,...","[0.345703125, -0.0279541015625, -0.03393554687...",False,"def fib(cv=1, lv=0):\n """"""The fibonacci seq...",return cv + lv,1
...,...,...,...,...,...,...,...,...,...,...,...,...
17305,mopidy.config.types.Float.deserialize,mopidy.config.types.Float.deserialize_5,151,"```Python\n def deserialize(self, value):\n...","[-0.1376953125, 0.5, -0.193359375, -0.09765625...","[-0.091796875, 0.146484375, 0.049072265625, -0...","[-0.08203125, 0.14453125, -0.08984375, 0.08593...","[0.126953125, -0.16796875, -0.107421875, 0.088...",False,"def deserialize(self, value):\n value ...",value = decode(value)\n validat...,1
17306,mopidy.config.types.Float.deserialize,mopidy.config.types.Float.deserialize_6,94,"```Python\n def deserialize(self, value):\n...","[-0.1376953125, 0.5, -0.193359375, -0.09765625...","[-0.08056640625, 0.515625, -0.01318359375, -0....","[-0.08203125, 0.14453125, -0.08984375, 0.08593...","[0.126953125, -0.16796875, -0.107421875, 0.088...",False,"def deserialize(self, value):\n value ...",value = decode(value)\n validat...,1
17307,mopidy.config.types.Float.deserialize,mopidy.config.types.Float.deserialize_7,191,"```Python\n def deserialize(self, value):\n...","[-0.1376953125, 0.5, -0.193359375, -0.09765625...","[-0.0751953125, 0.1103515625, 0.04150390625, -...","[-0.08203125, 0.14453125, -0.08984375, 0.08593...","[0.126953125, -0.16796875, -0.107421875, 0.088...",False,"def deserialize(self, value):\n value ...",value = decode(value)\n validat...,1
17308,mopidy.config.types.Float.deserialize,mopidy.config.types.Float.deserialize_8,163,"```Python\n def deserialize(self, value):\n...","[-0.1376953125, 0.5, -0.193359375, -0.09765625...","[-0.078125, 0.09912109375, 0.02587890625, -0.0...","[-0.08203125, 0.14453125, -0.08984375, 0.08593...","[0.126953125, -0.16796875, -0.107421875, 0.088...",False,"def deserialize(self, value):\n value ...",value = decode(value)\n v

In [3]:
import json

fail_namespaces = []

with open('/drive2/tuandung/DevEval/failed_samples.jsonl') as f:
    for line in f:
        data = json.loads(line)
        fail_namespaces.append(data['namespace'])

In [5]:
datasets = []
with open('/drive2/tuandung/DevEval/data.jsonl') as f:
    for line in f:
        data = json.loads(line)
        if data['namespace'] not in fail_namespaces:
            datasets.append(data)
len(datasets)

1733

In [6]:
with open('/drive2/tuandung/DevEval/data_clean.jsonl', 'w') as f:
    for data in datasets:
        f.write(json.dumps(data) + '\n')

In [5]:
import json

benchmark_data = []
project_data = {}
with open('/drive2/tuandung/DevEval/data.jsonl', 'r') as f:
    for line in f:
        js = json.loads(line)
        benchmark_data.append(js)
        project_name = js['completion_path'].split('/')[0]
        if project_name not in project_data:
            project_data[project_name] = []
        project_data[project_name].append(js)

import os
source_code_root = '/drive2/tuandung/DevEval/Source_Code'

ground_truth_completions = []
for data in benchmark_data:
    sos, eos = data['body_position'][0]-1, data['body_position'][1]
    completion_path = os.path.join(source_code_root, data['completion_path'])
    with open(completion_path, 'r') as f:
        file_lines = f.readlines()
    # file_lines = file_lines[:sos] + ['\n', completion, '\n'] + file_lines[eos:]
    ground_truth = ''.join(file_lines[sos:eos])
    ground_truth_completions.append({
        'namespace': data['namespace'],
        'completion': ground_truth  
    })

with open('ground_truth_completions.jsonl', 'w') as f:
    for js in ground_truth_completions:
        f.write(json.dumps(js) + '\n')